In [ ]:
from glob import glob
import random
import math
import os
import cv2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plot
import gc
from random import sample 
import multiprocess as mp
from functools import partial

In [ ]:
def selecting_images_preprocessing(images_path_array, limit_image_to_process="MAX", limit_image_to_train = "MAX", middle_rows=False):
    # images_path_array = glob(images_path)
    final_image_path = []
    def processing_image(img_path):
        image = cv2.imread(img_path)
        # print(image)
        mean = np.mean(image)
        std = np.std(image)
        # print(mean, image.mean())
        # print(std, image.std())
        data_row = {
            "image_path": img_path,
            "mean": image.mean(),
            "std": image.std(),
            # "class": 0
        }
        return data_row
    original_number_number_image = len(images_path_array)
    
    print("original number of data: ", original_number_number_image)
    
    if limit_image_to_train == "MAX":
        limit_image_to_train = original_number_number_image
    
    
    if len(images_path_array) < limit_image_to_process:
        print("The amount of dataset smaller than limit so we will use all images in dataset.")
    elif limit_image_to_process == "MAX":
        print("You choose to use all of data. please wait it will take a moment.")
    else:
        images_path_array = sample(images_path_array,limit_image_to_process)
    print("processed number of data: ", len(images_path_array))
    
    df_analysis = pd.DataFrame(columns=['image_path','mean','std'])
    counter = 0
    
    # multiple processing
    pool = mp.Pool(5)
    data_rows = pool.map(processing_image, images_path_array)
    
    # print(data_rows)
    
    df_analysis = df_analysis.append(data_rows, ignore_index = True)
    # counter += 1
    # if counter % 100 == 0:
    #     print("processed image: ", counter)
            
    final_df = df_analysis.sort_values(['std', 'mean'], ascending = [True, False])
    
    
    if middle_rows:
        n = len(final_df.index)
        mid_n = round(n/2)
        mid_k = round(limit_image_to_train/2)


        start = mid_n - mid_k
        end = mid_n + mid_k

        final = final_df.loc[start:end]
        final_image_path = final['image_path'].head(limit_image_to_train).tolist()
    else:
        final_image_path = final_df['image_path'].head(limit_image_to_train).tolist()
    
    
    # clear zombies memory
    del [[final_df, df_analysis]]
    gc.collect()
    
    return final_image_path

In [ ]:
# images_path = glob('mura_data/RGB/mura_march_clean/train_data/normal/*.png')
# test = selecting_images_preprocessing(images_path, 100, 10)
# print(test)

In [ ]:
images_path_array = glob('mura_data/RGB/mura_march_clean/train_data/normal/*.png') 
limit_image_to_process = 100
limit_image_to_train = 10
final_image_path = []
middle_rows = False



# images_path_array = glob(images_path)
def processing_image(img_path):
    image = cv2.imread(img_path)
    # print(image)
    mean = np.mean(image)
    std = np.std(image)
    # print(mean, image.mean())
    # print(std, image.std())
    data_row = {
        "image_path": img_path,
        "mean": image.mean(),
        "std": image.std(),
        # "class": 0
    }

    
    return data_row

In [ ]:
original_number_number_image = len(images_path_array)

print("original number of data: ", original_number_number_image)

if limit_image_to_train == "MAX":
    limit_image_to_train = original_number_number_image


if limit_image_to_process == "MAX":
    print("You choose to use all of data. please wait it will take a moment.")
elif len(images_path_array) < limit_image_to_process:
    print("The amount of dataset smaller than limit so we will use all images in dataset.")
else:
    images_path_array = sample(images_path_array,limit_image_to_process)
print("processed number of data: ", len(images_path_array))

df_analysis = pd.DataFrame(columns=['image_path','mean','std'])



pool = mp.Pool(5)


data_rows = pool.map(processing_image, images_path_array)
df_analysis = df_analysis.append(data_rows, ignore_index = True)

df_analysis.head(10)

In [ ]:
final_df = df_analysis.sort_values(['std', 'mean'], ascending = [True, False])
final_df.head()

In [ ]:
# counter += 1
# if counter % 100 == 0:
#     print("processed image: ", counter)
if middle_rows:
    print("Middle row")
    n = len(final_df.index)
    mid_n = round(n/2)
    mid_k = round(limit_image_to_train/2)


    start = mid_n - mid_k
    end = mid_n + mid_k

    final = final_df.loc[start:end]
    final_image_path = final['image_path'].head(limit_image_to_train).tolist()
else:
    print("Top row")
    final_image_path = final_df['image_path'].head(limit_image_to_train).tolist()
    
print(final_image_path)
print(len(final_image_path))
# clear zombies memory
# del [[final_df,df_analysis]]
# gc.collect()
# print(final_image_path)